In [1]:
pip install pymilvus



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install milvus-cli


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pymilvus import connections
from pymilvus import connections

# If using Docker standalone Milvus
connections.connect("default", host="127.0.0.1", port="19530")

In [4]:
from pymilvus import db


db.create_database("rag_db")




2025-11-11 19:24:59,524 [ERROR][handler]: RPC error: [create_database], <MilvusException: (code=65535, message=database already exist: rag_db)>, <Time:{'RPC start': '2025-11-11 19:24:59.519286', 'RPC error': '2025-11-11 19:24:59.524923'}> (decorators.py:253)


MilvusException: <MilvusException: (code=65535, message=database already exist: rag_db)>

In [5]:
from pymilvus import db 
db.using_database("rag_db")

In [6]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

# 3. Define schema for legal_docs collection

# Fields: doc_id, title, jurisdiction, date, embedding
fields = [
    FieldSchema(name="doc_id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=256),
    FieldSchema(name="jurisdiction", dtype=DataType.VARCHAR, max_length=64),
    FieldSchema(name="date", dtype=DataType.INT64),  # e.g., YYYYMMDD format
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=768)]
schema = CollectionSchema(fields, description="Legal document embeddings with metadata")

# Create collection
collection = Collection("legal_docs_2", schema)

In [7]:
import numpy as np

# Example embeddings (in reality, generate from a model like OpenAI/LegalBERT)
embedding1 = np.random.rand(768).tolist()
embedding2 = np.random.rand(768).tolist()
print(f"Embedding1 Length: {len(embedding1)}")
print(f"Embedding2 Length: {len(embedding2)}")  
print(f"Embedding1 Sample: {embedding1[:5]}")  # Print first 5 elements
print(f"Embedding2 Sample: {embedding2[:5]}")  # Print first 5 elements

docs = [
    [1, 2],  # doc_id
    ["Employment Law Case", "Tax Dispute"],  # title
    ["California", "New York"],  # jurisdiction
    [20230901, 20230715],  # date
    [embedding1, embedding2]  # embeddings
]

Embedding1 Length: 768
Embedding2 Length: 768
Embedding1 Sample: [0.7027637668445029, 0.2862503065786649, 0.847769580191779, 0.2936903563561917, 0.4761179101969364]
Embedding2 Sample: [0.010438552326913597, 0.12401136586240791, 0.849286938181681, 0.2741353501094086, 0.8656513704919779]


In [8]:
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {"nlist": 128}
}
collection.create_index(
    field_name="embedding",
    index_params=index_params
)

Status(code=0, message=)

In [9]:
collection.insert(docs)
collection.flush()

In [10]:
collection.load()

In [11]:
query_vector = embedding1  # pretend we want similar to doc 1

# Search only in California jurisdiction
results = collection.search(
    data=[query_vector],
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=3,
    expr='jurisdiction == "California"',
    output_fields=["doc_id", "title", "jurisdiction", "date"]
)

for res in results[0]:
    print(f"doc_id={res.entity.get('doc_id')}, "
          f"title={res.entity.get('title')}, "
          f"jurisdiction={res.entity.get('jurisdiction')}, "
          f"score={res.distance}")

doc_id=1, title=Employment Law Case, jurisdiction=California, score=1.0
